In [32]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math
import copy
import os
from datetime import datetime

In [34]:
df = pd.DataFrame([]) # 데이터프레임 만들기
df = pd.read_csv('../data/Data_Final_SeqDone.csv') 

df['Code'] = df['Code'].astype('category')

df['Date_Time'] = pd.to_datetime(df['Date_Time'].str.strip(), format="%Y-%m-%d %H:%M:%S", errors="raise")

df.head()

,Code,Blood_Glucose,Patient_ID,Code_Description,Date_Time
0,58,101.0,31,PRE_BREAKFAST,1991-05-20 08:00:00
1,33,5.0,31,REGULAR_INSULIN_DOSE,1991-05-20 12:00:00
2,34,27.0,31,NPH_INSULIN_DOSE,1991-05-20 08:00:00
3,60,89.0,31,PRE_LUNCH,1991-05-20 12:00:00
4,33,3.0,31,REGULAR_INSULIN_DOSE,1991-05-20 12:00:00


In [35]:
plt.rc("font", family="Malgun Gothic") # 한글 출력 적용

group_df = df.groupby('Patient_ID')

df_agg = pd.DataFrame({
    'median' : group_df['Blood_Glucose'].median(),
    
    'highs_perday' : group_df.apply(lambda group : group[group['Blood_Glucose']>180]['Blood_Glucose'].count()
                                    /(group.Date_Time.max() - group.Date_Time.min()).days),
    
    'lows_perday' : group_df.apply(lambda group : group[group['Blood_Glucose']<60]['Blood_Glucose'].count()
                                    /(group.Date_Time.max() - group.Date_Time.min()).days),
    
    'hypos_perday' : group_df.apply(
        lambda group : group[group['Code_Description']=='HYPOGLYCEMIC_SYMPTOMS']['Code_Description'].count()
        /((group.Date_Time.max() - group.Date_Time.min()).days)),
    
    'exercise_perday' : group_df.apply(
        lambda group : group[(group['Code_Description']=='TYPICAL_EXERCISE') |
                            (group['Code_Description']=='LESS_EXERCISE') |
                            (group['Code_Description']=='MORE_EXERCISE')]['Code_Description'].count()
        /((group.Date_Time.max() - group.Date_Time.min()).days)),
    
    'irregular_diet_perday' : group_df.apply(
        lambda group : group[(group['Code_Description']=='MORE_MEAL_INJESTION') |
                            (group['Code_Description']=='LESS_MEAL_INJESTION')]['Code_Description'].count()
        /((group.Date_Time.max() - group.Date_Time.min()).days)),
    
    'irregular_exercise_perday' : group_df.apply(
        lambda group : group[(group['Code_Description']=='LESS_EXERCISE') |
                            (group['Code_Description']=='MORE_EXERCISE')]['Code_Description'].count()
        /((group.Date_Time.max() - group.Date_Time.min()).days)),
    
    'readings_perday' : group_df['Date_Time'].apply(lambda group : group.count()
                                                /((group.max() - group.min()).days)),
    
    'snacks_perday' : group_df.apply(lambda group : group[group['Code_Description']=='PRE_SNACK']['Code_Description'].count()
                                    /(group.Date_Time.max() - group.Date_Time.min()).days),
    
    'regular_insulin_perday' : group_df.apply(lambda group : group[group['Code_Description']=='REGULAR_INSULIN_DOSE']['Code_Description'].count()
                                    /(group.Date_Time.max() - group.Date_Time.min()).days),
    
    'nph_insulin_perday' : group_df.apply(lambda group : group[group['Code_Description']=='NPH_INSULIN_DOSE']['Code_Description'].count()
                                    /(group.Date_Time.max() - group.Date_Time.min()).days),
    
    'ultralente_insulin_perday' : group_df.apply(lambda group : group[group['Code_Description']=='ULTRALENTE_INSULIN_DOSE']['Code_Description'].count()
                                    /(group.Date_Time.max() - group.Date_Time.min()).days),
    
    'insulin_perday' : group_df.apply(
        lambda group : group[(group['Code_Description']=='REGULAR_INSULIN_DOSE') |
                            (group['Code_Description']=='NPH_INSULIN_DOSE') |
                            (group['Code_Description']=='ULTRALENTE_INSULIN_DOSE')]['Code_Description'].count()
        /((group.Date_Time.max() - group.Date_Time.min()).days)),
    
    'postmeal_median_bg': group_df.apply(
        lambda group : group[(group['Code_Description']=='POST_LUNCH')]['Blood_Glucose'].median())
})

df_agg.loc[df_agg['postmeal_median_bg'].isnull(), 'postmeal_median_bg'] = df_agg.loc[df_agg['postmeal_median_bg'].isnull(), 'median']
# 결측값은 중앙값으로 채워준다,

df_agg.head(10)

,median,highs_perday,lows_perday,hypos_perday,exercise_perday,irregular_diet_perday,irregular_exercise_perday,readings_perday,snacks_perday,regular_insulin_perday,nph_insulin_perday,ultralente_insulin_perday,insulin_perday,postmeal_median_bg
Patient_ID,,,,,,,,,,,,,,
1,15.0,0.970370,4.103704,0.362963,0.000000,0.000000,0.000000,6.666667,0.000000,2.703704,0.970370,0.000000,3.674074,15.0
2,17.0,2.084211,4.042105,0.000000,0.000000,0.000000,0.000000,7.684211,0.000000,3.905263,0.021053,0.000000,3.926316,17.0
3,9.0,0.894737,4.184211,0.078947,0.000000,0.263158,0.000000,7.631579,0.394737,1.552632,0.026316,1.842105,3.421053,178.0
4,9.0,1.578947,4.131579,0.026316,0.000000,0.157895,0.000000,7.631579,0.842105,1.789474,1.421053,0.473684,3.684211,187.5
5,9.5,1.425000,3.900000,0.000000,0.000000,0.050000,0.000000,7.250000,0.900000,1.725000,1.875000,0.000000,3.600000,166.0
6,18.0,1.625000,3.500000,0.000000,0.000000,0.000000,0.000000,6.166667,0.000000,2.458333,1.041667,0.000000,3.500000,18.0
7,18.0,1.636364,3.878788,0.000000,0.000000,0.000000,0.000000,7.030303,0.000000,2.727273,1.000000,0.000000,3.727273,18.0
8,134.0,2.028571,1.228571,0.428571,0.057143,0.228571,0.000000,5.000000,0.457143,0.200000,0.114286,0.000000,0.314286,227.0
9,15.0,1.423077,4.500000,0.000000,0.115385,0.038462,0.076923,7.807692,0.884615,3.000000,0.846154,0.000000,3.846154,184.5


In [36]:
corr = np.absolute(df_agg[df_agg['hypos_perday']>0.0].corr())
# 저혈당 환자풀에서 각 데이터 간의 상관계수 생성

corr.sort_values(by='hypos_perday', ascending=False).hypos_perday
# hypos_perday를 기준으로 해서 hypos_perday(저혈당)에 영향을 주는 요인들을 영향력이 높은 순으로 내림차순 정렬


hypos_perday                 1.000000
irregular_diet_perday        0.532303
ultralente_insulin_perday    0.373580
exercise_perday              0.266994
postmeal_median_bg           0.237423
lows_perday                  0.232032
snacks_perday                0.219801
nph_insulin_perday           0.210841
irregular_exercise_perday    0.207763
median                       0.186336
readings_perday              0.127507
highs_perday                 0.121439
regular_insulin_perday       0.016155
insulin_perday               0.000141
Name: hypos_perday, dtype: float64

In [37]:
corr = np.absolute(df_agg.corr())
# 모든 환자풀에서 각 데이터 간의 상관계수 생성

corr.sort_values(by='lows_perday', ascending=False).lows_perday
# hypos_perday를 기준으로 해서 hypos_perday(저혈당)에 영향을 주는 요인들을 영향력이 높은 순으로 내림차순 정렬

lows_perday                  1.000000
readings_perday              0.942780
insulin_perday               0.842515
regular_insulin_perday       0.774709
median                       0.717307
exercise_perday              0.441693
nph_insulin_perday           0.420584
highs_perday                 0.405547
irregular_diet_perday        0.292857
hypos_perday                 0.269079
ultralente_insulin_perday    0.257409
irregular_exercise_perday    0.238126
snacks_perday                0.209366
postmeal_median_bg           0.073698
Name: lows_perday, dtype: float64